# 추천 시스템(Recommendation System)

- 콘텐츠 기반 필터링
- 협업 필터링 : NN(최근접 이웃), 잠재요인 -->우리는 이것을 사용!

In [1]:
#SGD 이용해 행렬 분해 수행

In [2]:
import numpy as np

#원본 행렬 R, 분해 행렬 P와 Q, 잠재 요인 차원 K=3
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
             [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
             [5, 2, 1, 2, np.NaN]]) # 4x5 행렬
num_users, num_items = R.shape
K=3 #k=3이므로, 행렬 분해는 4x5 = (4x3) * (5x3).T

In [3]:
import pandas as pd
pd.DataFrame(R)

,0,1,2,3,4
0,4.0,NaN,NaN,2.0,NaN
1,NaN,5.0,NaN,3.0,1.0
2,NaN,NaN,3.0,4.0,4.0
3,5.0,2.0,1.0,2.0,NaN


In [4]:
print(R.shape)

(4, 5)


In [5]:
num_users

4

In [6]:
#P와 Q 행렬의 크기를 지정하고, 정규 분포를 가진 임의의 값으로 입력 
#P, Q를 초기화함.(다음은 P와 Q의 초기값)
np.random.seed(1)
P=np.random.normal(scale=1./K, size=(num_users, K))
Q=np.random.normal(scale=1./K, size=(num_items, K))

print(P) #4x3
print(Q) #5x3 , 이건 곧 전치해서 3x5가 될것임.

[[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]]
[[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]


In [7]:
#get_rmse() : 실제 R과 예측행렬 R_hat의 차이에 대한 오차값?

from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error=0
    full_pred_matrix=np.dot(P, Q.T) #예측 행렬 R_hat
    
    #이 밑은, 실제 R 행렬은 NaN을 포함하고 있는데, 이 값이 없는 위치 인덱스를 추출해서 
    #실제 R 행렬과 예측 R_hat 행렬의 RMSE를 추출하는 코드
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind] #NAN 없는 R행렬
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind] #NAN 없는 R_hat 행렬
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [8]:
#SGD 기반으로 행렬 분해

non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0 ] #R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [9]:
steps = 1000 #SGD를 반복해서 업데이트할 횟수
learning_rate = 0.01 #SGD의 학습률
r_lambda = 0.01 #L2 정규화의 계수(람다)

#SGD 기법으로 P와 Q를 계속 업데이트하자.
for step in range(steps):
    for i, j, r in non_zeros: 
        eij = r-np.dot(P[i,:],Q[j,:].T) #실제 R - 예측 행렬 R_hat의 차
        P[i,:] = P[i, : ] + learning_rate*(eij*Q[j, :]-r_lambda*P[i,:])
        Q[j,:] = Q[j, : ] + learning_rate*(eij*P[i, :]-r_lambda*Q[j,:])
        rmse=get_rmse(R, P, Q, non_zeros) #앞에서 만든 rmse 함수. 오차값이 얼마가 나오는지?
        if (step % 50)==0 : #너무 steps가 많으니까.. 50으로 나눠떨어지는 step에 대해서만 값을 출력해보자.
            print("#### iteration step: ", step, " rmse : ", rmse) #P, Q의 값을 업데이트 시킬 때마다, rmse의 값이 확확 줄어드는 것을 볼 수 있다!

#### iteration step:  0  rmse :  3.261355059488935
#### iteration step:  0  rmse :  3.26040057174686
#### iteration step:  0  rmse :  3.253984404542389
#### iteration step:  0  rmse :  3.2521583839863624
#### iteration step:  0  rmse :  3.252335303789125
#### iteration step:  0  rmse :  3.251072196430487
#### iteration step:  0  rmse :  3.2492449982564864
#### iteration step:  0  rmse :  3.247416477570409
#### iteration step:  0  rmse :  3.241926055455223
#### iteration step:  0  rmse :  3.2400454107613084
#### iteration step:  0  rmse :  3.240166740749792
#### iteration step:  0  rmse :  3.2388050277987723
#### iteration step:  50  rmse :  0.5003190892212748
#### iteration step:  50  rmse :  0.5001616291326989
#### iteration step:  50  rmse :  0.49899601202578087
#### iteration step:  50  rmse :  0.4988483450145831
#### iteration step:  50  rmse :  0.49895189256631756
#### iteration step:  50  rmse :  0.49833236830090993
#### iteration step:  50  rmse :  0.4984148489378701
#### iterat

#### iteration step:  650  rmse :  0.016782895588885082
#### iteration step:  650  rmse :  0.016645698091647773
#### iteration step:  650  rmse :  0.01656714079916223
#### iteration step:  650  rmse :  0.016680091021598568
#### iteration step:  650  rmse :  0.016698554271430792
#### iteration step:  650  rmse :  0.016511017732427972
#### iteration step:  650  rmse :  0.016491228766905293
#### iteration step:  650  rmse :  0.01618054419796173
#### iteration step:  650  rmse :  0.01631111150707529
#### iteration step:  650  rmse :  0.01639316772050061
#### iteration step:  650  rmse :  0.01664473691247669
#### iteration step:  700  rmse :  0.0166383624426085
#### iteration step:  700  rmse :  0.016741936743323586
#### iteration step:  700  rmse :  0.016603524189001625
#### iteration step:  700  rmse :  0.016526454393300468
#### iteration step:  700  rmse :  0.016639792083379498
#### iteration step:  700  rmse :  0.016657201345297346
#### iteration step:  700  rmse :  0.016472928381641428

In [10]:
#이만하면 업데이트 많이 했다! 분해된 P와 Q 함수로 예측 행렬을 만들어 보자.

pred_matrix = np.dot(P, Q.T)
print("예측 행렬:\n", np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [11]:
#원본인 R과 값을 비교해 보자.
##값이 있는 값은 예측 행렬에서의 값과 비슷하고, NaN으로 된 값들도 예측 행렬에 의해 만든 값으로 채워넣으면 된다.
R 

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])

# 1. 협업 필터링 중 NN(아이템 기반) : 영화 간 유사도를 기반으로!

In [12]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [13]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
#ratings 데이터프레임을 user(행)/item(열)/rating(value)의 형태를 가진 행렬 R로 바꾸자. :pivot_table()
 
ratings=ratings[['userId', 'movieId', 'rating']] #'timestamp' 컬럼 드롭
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head() #역시 예상대로 NaN 값 짱많다. 원래 이럼. 사용자가 모든 영화에 대해 평점을 매기지 않기 때문
#user는 611명!

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#movieId 말고, title 이름으로 변경하자.(이것은 'movies'데이터프레임에 존재->merge를 통해 가능)
rating_merge = pd.merge(ratings, movies, on='movieId')

#columns='title'로 해서, 다시 피벗 테이블을 수행하자.
ratings_matrix2=rating_merge.pivot_table('rating', index='userId', columns='title')

#NaN 값은 모두 0으로 변환하자.
ratings_matrix2=ratings_matrix2.fillna(0)

ratings_matrix2 #컬럼네임과 NaN 값이 모두 바뀌었다!

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:
#영화 간 유사도 산출 : 코사인 유사도 기반 cosine_similarity()
#우리는 아이템 기반(item-based) NN을 수행하는 것임. 위의 cosine_similarity는 행 간의 유사도를 산출하므로 아이템 (영화)간 유사도를 산출하려면 
#위의 ratings_matrix2를 전치시켜주자.

ratings_matrix_T=ratings_matrix2.transpose()
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#자, 이제 각 행 간 코사인 유사도를 구해보자.(조금 시간이 걸린다)

from sklearn.metrics.pairwise import cosine_similarity

item_sim=cosine_similarity(ratings_matrix_T, ratings_matrix_T)

#반환된 넘파이 행렬을 영화 제목으로 매핑해 df 반환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix2.columns, columns=ratings_matrix2.columns) #item_sim의 행,열 이름은 전부 title임
print(item_sim_df.shape)
item_sim_df.head()

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot (2004),0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You (1997),0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [19]:
#그럼 만들어진 df를 참고하여, "Godfather, The (1972)" 이 영화와 유사도가 높은 영화 10개를 추출해 보자.

item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:10]

title
Godfather, The (1972)                                    1.000000
Godfather: Part II, The (1974)                           0.821773
Goodfellas (1990)                                        0.664841
One Flew Over the Cuckoo's Nest (1975)                   0.620536
Star Wars: Episode IV - A New Hope (1977)                0.595317
Fargo (1996)                                             0.588614
Star Wars: Episode V - The Empire Strikes Back (1980)    0.586030
Fight Club (1999)                                        0.581279
Reservoir Dogs (1992)                                    0.579059
Pulp Fiction (1994)                                      0.575270
Name: Godfather, The (1972), dtype: float64

In [20]:
#이번엔 "Inception"과 유사도가 높은 영화 10개!

item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:11] #0번은 자기 자신이 나와서..

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Interstellar (2014)              0.608150
Up (2009)                        0.606173
Avengers, The (2012)             0.586504
Django Unchained (2012)          0.581342
Departed, The (2006)             0.580849
Name: Inception (2010), dtype: float64

## 1-2) 영화 간 유사도뿐만 아니라 + 개인에게 맞춤형 추천 알고리즘으로 만들어 보자.

In [21]:
#예측 평점(R_hat)을 구하는 함수: predic_rating() -공식적용한 것!
def predict_rating(ratings_array, item_sim_array):
    ratings_pred = ratings_array.dot(item_sim_array) / np.array([np.abs(item_sim_array).sum(axis=1)])
    return ratings_pred

In [22]:
#함수적용
ratings_pred = predict_rating(ratings_matrix2.values, item_sim_df.values)

#df변환
ratings_pred_matrix= pd.DataFrame(ratings_pred, index=ratings_matrix2.index, columns=ratings_matrix2.columns) #행: userId, 열: tittle

ratings_pred_matrix.head() #구해진 예측 평점(R_hat)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501
4,0.049145,0.277628,0.160448,0.206892,0.309632,0.042337,0.130048,0.116442,0.099785,0.097432,...,0.051269,0.076051,0.055563,0.054137,0.008343,0.159242,0.100941,0.062253,0.146054,0.231187
5,0.007278,0.066951,0.041879,0.013880,0.024842,0.018240,0.026405,0.018673,0.021591,0.018841,...,0.009689,0.022246,0.013360,0.012378,0.000000,0.025839,0.023712,0.018012,0.028133,0.052315


In [23]:
#MSE는?
from sklearn.metrics import mean_squared_error

#사용자가 평점을 부여한 영화에 대해서만(R에 값이 있는 것만-> actual.nonzero()) 예측 성능 평가 MSE를 구함.
def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('MSE는: ', get_mse(ratings_pred, ratings_matrix2.values))

MSE는:  9.895354759094706


- 앞의 MSE를 줄이기 위해 TOP-N 유사도 벡터를 다시 만든다.

In [24]:
#앞의 predict_rating() 함수는 모든 영화 평점 간에 유사도 벡터를 적용해서 MSE가 높은 편이다.
#MSE를 줄이기 위해 -> TOP-N 유사도를 가지는 영화 유사도 벡터만 적용해보자.(특정 영화와 유사도가 높은 N개에 대해서만 유사도 벡터를 적용!)
#predict_rating_topsim() 함수. 단점이라면 시간이 좀 오래 걸린다는 거..(2분쯤?)

def predict_rating_topsim(ratings_array, item_sim_array, n=20): #여기서 N=20
    #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬로 초기화
    pred = np.zeros(ratings_array.shape)
    
    #사용자-아이템 평점 행렬의 열 크기만큼 루프 돌림
    for col in range(ratings_array.shape[1]):
        top_n_items = [np.argsort(item_sim_array[:, col])[:-n-1:-1]] #item_sim_array 에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        #개인화된 예측 평점은
        for row in range(ratings_array.shape[0]):
            pred[row, col] = item_sim_array[col, :][top_n_items].dot(ratings_array[row, :][top_n_items].T) / np.sum(np.abs(item_sim_array[col, :][top_n_items]))
            
    return pred

In [25]:
#함수 적용
ratings_pred = predict_rating_topsim(ratings_matrix2.values, item_sim_df.values, n=20)
#mse 계산
print(get_mse(ratings_pred, ratings_matrix2.values)) #mse가 많이 줄은 것을 볼 수 있지?(9에서 3으로)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


3.69501623729494


In [26]:
#df변환
ratings_pred_matrix= pd.DataFrame(ratings_pred, index=ratings_matrix2.index, columns=ratings_matrix2.columns) #행: userId, 열: tittle

ratings_pred_matrix.head() #구해진 예측 평점(R_hat)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.220798,0.0,0.0,1.677291,0.284372
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.220798,0.0,0.0,0.194828,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [27]:
#그럼 이제 특정 사용자(userId:9번)가 어떤 영화를 좋아하는지 확인해 보자.
user_rating_id = ratings_matrix2.loc[9, : ] #유저아이디 9번이 준 평점목록
user_rating_id

title
'71 (2014)                                                                                  0.0
'Hellboy': The Seeds of Creation (2004)                                                     0.0
'Round Midnight (1986)                                                                      0.0
'Salem's Lot (2004)                                                                         0.0
'Til There Was You (1997)                                                                   0.0
'Tis the Season for Love (2015)                                                             0.0
'burbs, The (1989)                                                                          0.0
'night Mother (1986)                                                                        0.0
(500) Days of Summer (2009)                                                                 0.0
*batteries not included (1987)                                                              0.0
...All the Marbles (1981)         

In [28]:
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10] #이 평점이 0보다 큰 것 중에서 내림차순 정렬. 이런 영화를 좋아하는구나?

title
Adaptation (2002)                                                                 5.0
Austin Powers in Goldmember (2002)                                                5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Producers, The (1968)                                                             5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Back to the Future (1985)                                                         5.0
Glengarry Glen Ross (1992)                                                        4.0
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)                                     4.0
Name: 9, dtype: float64

- 이 유저아이디 9번에게 우리가 만든 코드를 이용해 영화를 추천해 주자.

In [29]:
#평점을 주지 않은 영화를 리스트로 반환하는 함수: get_unseen_movies()
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId, :]
    already_seen = user_rating[user_rating > 0].index.tolist() #일단 평점이 0보다 크면 이전에 봤다는 뜻
    
    movies_list = ratings_matrix.columns.tolist() #모든 영화명(title)을 list로
    
    unseen_list = [movie for movie in movies_list if movie not in already_seen] #mvies_list에 있는 것 중 already_seen에 없는 것들 반환
    
    return unseen_list

In [30]:
#영화 추천 함수: recomm_movie_by_userid()

def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n] #예측 행렬에서 유저아이디와, unseen_list에 있는 영화만 추출, 그 후 내림차순 정렬. 예측 평점이 높은 순서대로.
    return recomm_movies

- 이제 진짜로 영화 추천! 함수적용 ㄱㄱ.

In [31]:
unseen_list = get_unseen_movies(ratings_matrix2, 9) #유저아이디 9번이 관람하지 않은 영화명
unseen_list[:5]

["'71 (2014)",
 "'Hellboy': The Seeds of Creation (2004)",
 "'Round Midnight (1986)",
 "'Salem's Lot (2004)",
 "'Til There Was You (1997)"]

In [32]:
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)
recomm_movies
#여기서는 TOP-N 유사도가 적용된, rating_pred를 df 변환한 ratings_pred_matrix 를 pred_df의 인자로 넣었다.

title
Shrek (2001)                                                                                      0.866202
Spider-Man (2002)                                                                                 0.857854
Last Samurai, The (2003)                                                                          0.817473
Indiana Jones and the Temple of Doom (1984)                                                       0.816626
Matrix Reloaded, The (2003)                                                                       0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)    0.765159
Gladiator (2000)                                                                                  0.740956
Matrix, The (1999)                                                                                0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003)                                     0.689591
Lord of the Rings: The Return o

In [33]:
#df 변환
recomm_df= pd.DataFrame(recomm_movies.values, index= recomm_movies.index, columns = ['pred_score']) 
#꼭 데이터에 values 처리! 아님 NAN인식함. 컬럼명은 우리가 만들어준거임

In [34]:
recomm_df

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


In [35]:
#결과값을 보니, 슈렉을 가장 선호할 것 같군요.

# 2. 협업 필터링 중 잠재 요인 : 행렬분해 이용(SVD, NMF) + SGD

- NaN 값이 많아서 우리는 SGD를 이용한 SVD로 행렬분해를 할 것임!

In [36]:
def matrix_factorization(R, K, steps = 200, learning_rate =0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    #P와 Q 행렬의 크기를 지정하고, 정규 분포를 가진 임의의 값으로 입력 
    #P, Q를 초기화함.(다음은 P와 Q의 초기값)
    np.random.seed(1)
    P=np.random.normal(scale=1./K, size=(num_users, K))
    Q=np.random.normal(scale=1./K, size=(num_items, K))
    
    prev_rmse=10000
    break_count=0
    
    #SGD 기반으로 행렬 분해
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0 ] #R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
    
    #SGD 기법으로 P, Q를 계속 업데이트.(rmse가 작아지도록)
    for step in range(steps):
        for i, j, r in non_zeros: 
            eij = r-np.dot(P[i,:],Q[j,:].T) #실제 R - 예측 행렬 R_hat의 차
            P[i,:] = P[i, : ] + learning_rate*(eij*Q[j, :]-r_lambda*P[i,:])
            Q[j,:] = Q[j, : ] + learning_rate*(eij*P[i, :]-r_lambda*Q[j,:])
            rmse=get_rmse(R, P, Q, non_zeros) #앞에서 만든 rmse 함수. 오차값이 얼마가 나오는지?
            
            if (step % 10)==0:
                print("### iteration step: ", step, " rmse: ", rmse) 
            
    return P, Q

In [37]:
#앞에서 만들었던 ratings_matrix2 다시 가지고 오자

ratings_matrix2.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#ratings_matrix2를 matrix_factorization() 함수를 이용해 행렬 분해해 보자.(시간이 좀 걸린다. 특히 step 수가 많아질수록..)
#n_factor의 수 10개, step을 20으로 했을 때

P, Q = matrix_factorization(ratings_matrix2.values, K=10, steps= 20, learning_rate=0.01, r_lambda=0.01)

### iteration step:  0  rmse:  3.653659989810904
### iteration step:  0  rmse:  3.653659767406136
### iteration step:  0  rmse:  3.6536596689361778
### iteration step:  0  rmse:  3.6536595170280073
### iteration step:  0  rmse:  3.653659108394117
### iteration step:  0  rmse:  3.6536591502430267
### iteration step:  0  rmse:  3.6536587248390533
### iteration step:  0  rmse:  3.653658894806323
### iteration step:  0  rmse:  3.653659059226685
### iteration step:  0  rmse:  3.653659122408712
### iteration step:  0  rmse:  3.6536588338285934
### iteration step:  0  rmse:  3.6536591929721203
### iteration step:  0  rmse:  3.653659379928703
### iteration step:  0  rmse:  3.653659215229705
### iteration step:  0  rmse:  3.6536592569030115
### iteration step:  0  rmse:  3.653659062617725
### iteration step:  0  rmse:  3.6536588650816046
### iteration step:  0  rmse:  3.653658800441468
### iteration step:  0  rmse:  3.653659050661183
### iteration step:  0  rmse:  3.6536591089200825
### iterati

### iteration step:  0  rmse:  3.6536481709118087
### iteration step:  0  rmse:  3.6536480607835258
### iteration step:  0  rmse:  3.6536486367115035
### iteration step:  0  rmse:  3.653648628473399
### iteration step:  0  rmse:  3.6536485015934526
### iteration step:  0  rmse:  3.653648489878106
### iteration step:  0  rmse:  3.653648605065013
### iteration step:  0  rmse:  3.653648339383844
### iteration step:  0  rmse:  3.6536486103883408
### iteration step:  0  rmse:  3.653648599182146
### iteration step:  0  rmse:  3.6536485531170597
### iteration step:  0  rmse:  3.653648475657861
### iteration step:  0  rmse:  3.65364872500506
### iteration step:  0  rmse:  3.653648515120853
### iteration step:  0  rmse:  3.6536484817241193
### iteration step:  0  rmse:  3.653648483447435
### iteration step:  0  rmse:  3.653648682145364
### iteration step:  0  rmse:  3.653648497558835
### iteration step:  0  rmse:  3.6536482882002685
### iteration step:  0  rmse:  3.6536484423954394
### iteratio

### iteration step:  0  rmse:  3.6536429005835998
### iteration step:  0  rmse:  3.6536425284505447
### iteration step:  0  rmse:  3.653642524832848
### iteration step:  0  rmse:  3.6536429125866023
### iteration step:  0  rmse:  3.6536428523348157
### iteration step:  0  rmse:  3.653642862880051
### iteration step:  0  rmse:  3.6536429215737103
### iteration step:  0  rmse:  3.6536430103150526
### iteration step:  0  rmse:  3.6536430211547346
### iteration step:  0  rmse:  3.6536430515073084
### iteration step:  0  rmse:  3.6536431029707157
### iteration step:  0  rmse:  3.6536426498818475
### iteration step:  0  rmse:  3.6536425296082733
### iteration step:  0  rmse:  3.653642433812483
### iteration step:  0  rmse:  3.653642466119029
### iteration step:  0  rmse:  3.653642013898674
### iteration step:  0  rmse:  3.653641777962712
### iteration step:  0  rmse:  3.6536417885560093
### iteration step:  0  rmse:  3.653642095682985
### iteration step:  0  rmse:  3.653642245728024
### iter

### iteration step:  0  rmse:  3.6536384931177777
### iteration step:  0  rmse:  3.653638399214325
### iteration step:  0  rmse:  3.653638510432015
### iteration step:  0  rmse:  3.6536385346183207
### iteration step:  0  rmse:  3.65363757189148
### iteration step:  0  rmse:  3.6536375720948704
### iteration step:  0  rmse:  3.653637607486511
### iteration step:  0  rmse:  3.653637791075855
### iteration step:  0  rmse:  3.6536376259783814
### iteration step:  0  rmse:  3.65363750349476
### iteration step:  0  rmse:  3.6536375628534112
### iteration step:  0  rmse:  3.6536372939361335
### iteration step:  0  rmse:  3.653637496184717
### iteration step:  0  rmse:  3.6536380036736604
### iteration step:  0  rmse:  3.6536379838524393
### iteration step:  0  rmse:  3.6536381582710797
### iteration step:  0  rmse:  3.6536380990371984
### iteration step:  0  rmse:  3.65363795017627
### iteration step:  0  rmse:  3.6536376914347386
### iteration step:  0  rmse:  3.653637563069467
### iteratio

### iteration step:  0  rmse:  3.653626212141252
### iteration step:  0  rmse:  3.653626131650314
### iteration step:  0  rmse:  3.65362607674899
### iteration step:  0  rmse:  3.6536259621621374
### iteration step:  0  rmse:  3.6536259448696207
### iteration step:  0  rmse:  3.6536261278521023
### iteration step:  0  rmse:  3.6536260949292325
### iteration step:  0  rmse:  3.6536259596968454
### iteration step:  0  rmse:  3.6536261158422674
### iteration step:  0  rmse:  3.653625987799784
### iteration step:  0  rmse:  3.6536261292695733
### iteration step:  0  rmse:  3.6536261264320804
### iteration step:  0  rmse:  3.6536262131577066
### iteration step:  0  rmse:  3.6536262873442737
### iteration step:  0  rmse:  3.6536262167102866
### iteration step:  0  rmse:  3.6536261915442956
### iteration step:  0  rmse:  3.653626298012578
### iteration step:  0  rmse:  3.6536262634503376
### iteration step:  0  rmse:  3.6536265071933585
### iteration step:  0  rmse:  3.6536264217393213
### it

...

### iteration step:  0  rmse:  3.6511775183556017
### iteration step:  0  rmse:  3.6511773572871618
### iteration step:  0  rmse:  3.6511785363777665
### iteration step:  0  rmse:  3.651177769859694
### iteration step:  0  rmse:  3.6511771848903605
### iteration step:  0  rmse:  3.6511774785214706
### iteration step:  0  rmse:  3.6511774580738834
### iteration step:  0  rmse:  3.651177575883709
### iteration step:  0  rmse:  3.65117714612728
### iteration step:  0  rmse:  3.6511769734532407
### iteration step:  0  rmse:  3.6511769179299645
### iteration step:  0  rmse:  3.65117539194981
### iteration step:  0  rmse:  3.6511740836562367
### iteration step:  0  rmse:  3.6511734409934857
### iteration step:  0  rmse:  3.6511728564828396
### iteration step:  0  rmse:  3.6511733018360735
### iteration step:  0  rmse:  3.6511720991969194
### iteration step:  0  rmse:  3.651171400789495
### iteration step:  0  rmse:  3.6511690137517285
### iteration step:  0  rmse:  3.6511689626010115
### ite

### iteration step:  0  rmse:  3.651132521524002
### iteration step:  0  rmse:  3.651132263272483
### iteration step:  0  rmse:  3.6511320128456513
### iteration step:  0  rmse:  3.651131998814519
### iteration step:  0  rmse:  3.651132128432639
### iteration step:  0  rmse:  3.6511319843606413
### iteration step:  0  rmse:  3.6511312742031303
### iteration step:  0  rmse:  3.6511303746827575
### iteration step:  0  rmse:  3.6511303665955657
### iteration step:  0  rmse:  3.6511303614382613
### iteration step:  0  rmse:  3.651130482661202
### iteration step:  0  rmse:  3.651129905854922
### iteration step:  0  rmse:  3.651129923746041
### iteration step:  0  rmse:  3.6511300723470064
### iteration step:  0  rmse:  3.651129740902382
### iteration step:  0  rmse:  3.6511294770804823
### iteration step:  0  rmse:  3.651129237768192
### iteration step:  0  rmse:  3.6511290373166343
### iteration step:  0  rmse:  3.651128433718939
### iteration step:  0  rmse:  3.651127836579356
### iterati

### iteration step:  0  rmse:  3.6510981141290983
### iteration step:  0  rmse:  3.651098061611371
### iteration step:  0  rmse:  3.651098141231209
### iteration step:  0  rmse:  3.6510985406672476
### iteration step:  0  rmse:  3.651098193100169
### iteration step:  0  rmse:  3.651098197760703
### iteration step:  0  rmse:  3.6510982817858992
### iteration step:  0  rmse:  3.6510984593405333
### iteration step:  0  rmse:  3.6510983091153775
### iteration step:  0  rmse:  3.6510983621456563
### iteration step:  0  rmse:  3.6510984345698265
### iteration step:  0  rmse:  3.6510981320364624
### iteration step:  0  rmse:  3.651097593878079
### iteration step:  0  rmse:  3.651097829080049
### iteration step:  0  rmse:  3.651098065778202
### iteration step:  0  rmse:  3.6510975578911236
### iteration step:  0  rmse:  3.6510970054143463
### iteration step:  0  rmse:  3.6510968025372956
### iteration step:  0  rmse:  3.651096745043633
### iteration step:  0  rmse:  3.651096992103446
### itera

### iteration step:  0  rmse:  3.651086049375947
### iteration step:  0  rmse:  3.6510860770389644
### iteration step:  0  rmse:  3.65108545635707
### iteration step:  0  rmse:  3.651085159346264
### iteration step:  0  rmse:  3.65108531411715
### iteration step:  0  rmse:  3.651084499353267
### iteration step:  0  rmse:  3.651084621892921
### iteration step:  0  rmse:  3.651084568441591
### iteration step:  0  rmse:  3.651084538619292
### iteration step:  0  rmse:  3.651084521806705
### iteration step:  0  rmse:  3.6510844291438973
### iteration step:  0  rmse:  3.6510841091217374
### iteration step:  0  rmse:  3.6510844043613795
### iteration step:  0  rmse:  3.6510842798330736
### iteration step:  0  rmse:  3.6510837919407493
### iteration step:  0  rmse:  3.6510833521377797
### iteration step:  0  rmse:  3.6510833002551593
### iteration step:  0  rmse:  3.6510833487829393
### iteration step:  0  rmse:  3.6510830909488203
### iteration step:  0  rmse:  3.6510832015757844
### iterati

### iteration step:  0  rmse:  3.6510516162224698
### iteration step:  0  rmse:  3.6510511597808875
### iteration step:  0  rmse:  3.6510518488018016
### iteration step:  0  rmse:  3.65105142228343
### iteration step:  0  rmse:  3.651051463188988
### iteration step:  0  rmse:  3.651051544393924
### iteration step:  0  rmse:  3.6510515050701065
### iteration step:  0  rmse:  3.651050829854699
### iteration step:  0  rmse:  3.6510511240156673
### iteration step:  0  rmse:  3.651051079499714
### iteration step:  0  rmse:  3.6510505409879785
### iteration step:  0  rmse:  3.6510499117798756
### iteration step:  0  rmse:  3.6510493819752443
### iteration step:  0  rmse:  3.6510493455282087
### iteration step:  0  rmse:  3.651049753055556
### iteration step:  0  rmse:  3.6510492596230786
### iteration step:  0  rmse:  3.651049312430757
### iteration step:  0  rmse:  3.651049805947382
### iteration step:  0  rmse:  3.651049788694876
### iteration step:  0  rmse:  3.651049574635673
### iterati

In [ ]:
#그래서 반환된 예측 행렬(R_hat)은?
pred_matrix=np.dot(P, Q.T)
pred_matrix

In [ ]:
#pred_matrix를 df 로 변환
df_pred_matrix = pd.DataFrame(pred_matrix, index=ratings_matrix2.index, columns=ratings_matrix2.columns) #행: userId, 열: title
df_pred_matrix.head()

## 2-2) df_pred_matrix를 이용해, 개인화된 영화 추천을 해보자!

- 앞의 NN에서 했던 것과 마찬가지로 유저아이디 9번에게 영화 추천을 해주겠다. 또한 앞에서 만든 get_unseen_movies(), recomm_movie_by_userid() 함수 그대로 사용

In [ ]:
unseen_list = get_unseen_movies(ratings_matrix2, 9) #유저아이디 9번이 보지 않은 영화명
unseen_list

In [ ]:
#잠재 요인 협업 필터링으로 영화 추천
recomm_movies=recomm_movie_by_userid(df_pred_matrix, 9, unseen_list, top_n=10)
recomm_movies

In [ ]:
#df변환
df_recomm=pd.DataFrame(recomm_movies.value, index=recomm_movies.index, columns=['pred_score'])
df_recomm